# 1 finalize plots in TS analysis and save them to default shit
# 3 fix model stuff

In [156]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from pylab import rcParams
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn import metrics
import os
import glob
import json
import warnings
import itertools
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, kpss


warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')

class DataLoader():
    def __init__(self, datapath='./routes.csv', weatherdata='./enrichment/lima_2015_weatherdata.csv'):
        self.datapath = datapath
        assert os.path.exists(self.datapath)
        self.__setmissingpath("./cleaning")
        self.__setweatherpath(weatherdata)                     # additional data to enrich the dataset
        df = pd.read_csv("./routes.csv", sep='\t')
                              
        df = self.cleanData(df)
        self.__setData(df)
        self.__setResampledData__(self.mergeWeatherData(df))           # after cleaning & merging
    
    def getData(self):
        return self.__data
    
    def getResampledData(self):
        return self.__resampledData
                              
    def getmissingpath(self):
        return self.__missingpath
    
    def getweatherpath(self):
        return self.__weatherdata
   
    def __setData(self, df):
        self.__data = df

    def __setResampledData__(self, df):
        self.__resampledData = df
        
    def __setmissingpath(self, path):
        self.__missingpath = path
        
    def __setweatherpath(self, path):
        self.__weatherdata = path
        
    def cleanData(self, df):
        print("Cleaning the Data...")
        sns_plot = sns.heatmap(df.isnull(), yticklabels=False, cbar=True, cmap='viridis'
           #,vmax=1.0, vmin=-1.0   # these solve the boolean - problem
           )
        if not os.path.exists(self.getmissingpath()):
            os.mkdir(self.getmissingpath())
        fig = sns_plot.figure.savefig(self.getmissingpath()+"/missingdata.png")
        plt.close(fig)
        print("We have "+str(np.around(len(df[df.isnull().any(axis=1)])/len(df)*100, decimals=3))+"% missing values so we drop them")    #def loadWeatherData(self):
        df = df.dropna()
        return df
      
    def loadWeatherData(self):
        """
        Hourly sampled dataset with temperature, wind and rain information
        """
        print("Loading additional weather data...")
        assert os.path.exists(self.getweatherpath())
        weatherdf = pd.read_csv(self.getweatherpath())
        return weatherdf
        
    def mergeWeatherData(self, df):
        print("Enriching...")
        dfw = self.loadWeatherData()
        #Resample df to a frequency of an hour
        df["request_date"]= pd.to_datetime(df["request_date"]) 
        df = df.resample('h', on = 'request_date').count()
        df = df.rename(columns={'passenger_id': 'requests'})
        df = df[['requests']]

        # Reset Indexes to merge
        df = df.reset_index()  
        dfw = dfw.reset_index()
        
        dfw['datetime'] = pd.to_datetime(dfw['datetime'])
        df = df.merge(dfw, left_on='request_date', right_on='datetime', how='left')
        df = df.fillna(df.mean())                                        # impute 6 missing values with mean
        df.drop(columns=['datetime', 'index'], inplace=True)                      # remove 2nd index
        
        # try to reindex
        df = df.set_index('request_date')
        df = df.asfreq(freq='h')    
        
        #print("new size is "+str(len(df)))
        return df
        

class Cluster_Analysis():
    def __init__(self, df, clusterpath='./clustering', explore=False):
        self.__setClusterpath__(clusterpath) 
        self.__setData__(df)
        if explore!=False:
            self.clusterExploration(df, 'source')
            self.clusterExploration(df, 'destination')
        df = self.clusterData(df)
        self.drawClustersOnMap(df, 'source')
        self.drawClustersOnMap(df, 'destination')
    
    def getData(self):
        return self.__data
    
    def __setData__(self, df):
        self.__data = df
    
    def getClusterpath(self):
        return self.__clusterpath
                              
    def __setClusterpath__(self, path):
        self.__clusterpath = path 
                              
    def clusterExploration(self, df_geo, kind):
        performanceList = []
        K = range(5,30)
        """
        Trains K-means to the coordinates and exports elbow diagram to outputfolder
        Arguments:
            df: Input dataframe
            kind (string): 'source' or 'destination'
            outputfolder (string): folder to export elbow diagram
        Returns:
            data (pd.DataFrame): new dataframe with two new columns
            with the predicted source and destination based on the clustering
            algorithm
        """
        assert kind=='source' or kind=='destination'
        print("Exploring K for KMeans for "+str(kind)+" coordinates...")
        df_geo = df_geo[[str(kind)+'_longitude', str(kind)+'_longitude']]
        
        # Explore K
        for k in K:
            kmeanModel = KMeans(n_clusters=k, n_jobs=-1).fit(df_geo)
            #distortions.append(sum(np.min(cdist(df_geo, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / df_geo.shape[0])
            performanceList.append(kmeanModel.inertia_)
            
        # Plot the elbow
        plt.plot(K, performanceList, 'bx-')
        plt.xlabel('k')
        plt.ylabel('Sum of squared distances')
        plt.title('Error vs K in '+str(kind)+' coordinates')
        if not os.path.exists(self.clusterpath):
            os.mkdir(outputfolder)
        plt.savefig(self.clusterpath+'/elbow_rule_'+str(kind)+'.png')
        #plt.show()
        
    def clusterData(self, df, k=15):                                    # default after elbow rule
        print("Performing K-means for both source and destination coordinates with K="+str(k))
        kmeans_model = KMeans(n_clusters=k, n_jobs=-1).fit(df[['source_latitude', 'source_longitude']])
        df['sourceCluster'] = kmeans_model.predict(df[['source_latitude', 'source_longitude']])

        kmeans_model = KMeans(n_clusters=k, n_jobs=-1).fit(df[['destination_latitude', 'destination_longitude']])
        df['destinationCluster'] = kmeans_model.predict(df[['destination_latitude', 'destination_longitude']])
        return df

    def calculateBoarders(self, df, decimals=4):
        # Needed to download Lima map from https://www.openstreetmap.org/export#map=5/51.500/-0.100

        sourceDict = {'longitude_max': np.around(df['source_longitude'].max(), decimals=decimals), 
                      'longitude_min': np.around(df['source_longitude'].min(), decimals=decimals), 
                      'latitude_max': np.around(df['source_latitude'].max(), decimals=decimals), 
                      'latitude_min': np.around(df['source_latitude'].min(), decimals=decimals)}

        destinationDict = {'longitude_max': np.around(df['destination_longitude'].max(), decimals=decimals), 
                      'longitude_min': np.around(df['destination_longitude'].min(), decimals=decimals), 
                      'latitude_max': np.around(df['destination_latitude'].max(), decimals=decimals), 
                      'latitude_min': np.around(df['destination_latitude'].min(), decimals=decimals)}
        
        
    def drawClustersOnMap(self, df, kind, photopath='./limaMap.png'):
        assert (kind=='source') or (kind=='destination')
        assert os.path.exists(photopath)
    
        print("Generating Cluster Map for "+str(kind)+" coordinates...")
        clusters = len(df[str(kind)+'Cluster'].unique())
        x = np.arange(clusters+1)                         # used for different colors of clusters
        ys = [i+x+(i*x)**2 for i in range(clusters+1)]
        colors = matplotlib.cm.rainbow(np.linspace(0, 1, len(ys)))


        ruh_m = plt.imread(photopath)

        BBox = ((df[str(kind)+'_longitude'].min(),   
                 df[str(kind)+'_longitude'].max(),      
                 df[str(kind)+'_latitude'].min(), 
                 df[str(kind)+'_latitude'].max()))

        fig, ax = plt.subplots(figsize = (8,7))

        # For all Clusters

        num=0
        #for k,c in zip(df['pred_'+str(sd)].unique(), colors):
        for k in df[str(kind)+'Cluster'].unique():
            #num+=1

            ax.scatter(df[df[str(kind)+'Cluster']==k][str(kind)+'_longitude'], 
                       df[df[str(kind)+'Cluster']==k][str(kind)+'_latitude'], 
                       zorder=1, alpha= 0.2, c=[np.random.rand(3,)], s=10)                  # random color
            #print(num)

        ax.set_title('Plotting Spatial Data on Lima Map')
        ax.set_xlim(BBox[0],BBox[1])
        ax.set_ylim(BBox[2],BBox[3])
        ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal')
        fig.savefig(str(self.getClusterpath())+'/Clustermap_'+str(kind)+'.png')
        plt.close(fig)



class TS_Analysis():    # add export of plots
    def __init__(self, df, exportpath='./TS Decomposition/'):
        self.__setData__(df)
        self.__setExportPath__(exportpath)
        #self.plotTS()
        #self.TSDecomposition()  
        self.__setTestResults__(self.stationarityTests())
        
    def getExportPath(self):
        return self.__exportpath
        
    def __setData__(self, df):
        self.__data = df
        
    def __setTestResults__(self, df):
        self.__testResults = df
        
    def __setExportPath__(self, exportpath):
        self.__exportpath = exportpath
        
    def getData(self):
        return self.__data
    
    def getTestResults(self):
        return self.__testResults
                    
    def plotTS(self):
        df = self.getData()
        #df.reset_index(inplace=True)
        fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(15,15))
        figure = sns.lineplot(x='request_date', y='requests', color="indianred", data=dfl, ax=axs[0])
        for item in figure.get_xticklabels():
            item.set_rotation(45)
        figure = sns.lineplot(x='request_date', y='Temperature', color="blue", data=dfl, ax=axs[1])
        for item in figure.get_xticklabels():
            item.set_rotation(45)
        figure = sns.lineplot(x='request_date', y='Precipitation', color="green", data=dfl, ax=axs[2])
        for item in figure.get_xticklabels():
            item.set_rotation(45)
        figure = sns.lineplot(x='request_date', y='WindSpeed', color="black", data=dfl, ax=axs[3])
        for item in figure.get_xticklabels():
            item.set_rotation(45)
        plt.savefig(self.getExportPath()+"./plots.png")
        plt.close(fig)
                    
    def TSDecomposition(self):                                   # examine trend, seasonality
        rcParams['figure.figsize'] = 18, 8
        print("Exporting Time Series Decomposition...")
        usefulcols = ['requests', 'Temperature', 'Precipitation', 'WindSpeed']  
        # assess columns?
        df = self.getData()
        #df.reset_index(inplace=True)
        for col in usefulcols:                           # do it for all relevant TS columns
            temp = df[['request_date', col]]
            temp = temp.set_index('request_date')
            temp = temp.asfreq(freq='h')
            
            decomposition = sm.tsa.seasonal_decompose(temp, model='additive')
            fig = decomposition.plot()
            fig.savefig(self.getExportPath()+str(col)+'_decomposition.png')
            plt.close(fig)
            #plt.show()
    
    def kpss_test(self, timeseries):
        #print ('Results of KPSS Test:')
        kpsstest = kpss(timeseries, regression='c')
        kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
        for key,value in kpsstest[3].items():
            kpss_output['Critical Value (%s)'%key] = value
        #print (kpss_output)
        return kpss_output
    
    def adf_test(self, timeseries):    # The more negative the Test Statistic is, the harder we reject H0: unit root/stationary
        #Perform Dickey-Fuller test:                 # equally: H0: TS is non-stationary
        #print ('Results of Dickey-Fuller Test:')
        dftest = adfuller(timeseries, autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
        for key,value in dftest[4].items():
            dfoutput['Critical Value (%s)'%key] = value
        #print (dfoutput) 
        return dfoutput
    
    def stationarityTests(self):        # check for stationarity to determine whether its necessary to transform
        df = self.getData()             # (log, boxcox) for forecasting
        dftest = pd.DataFrame()
        for col in df.columns:                # return all results for both tests and all TS in a dataframe
            dfadf = self.adf_test(df[col]) 
            row = pd.Series({'H0 Rejected':True if dfadf.loc['p-value'] <= 0.05 else False})   # result of the test
            dfadf = dfadf.append(row)
            
            dfkpss = self.kpss_test(df[col])
            row = pd.Series({'H0 Rejected':True if dfkpss.loc['p-value'] <= 0.05 else False})  # result of the test
            dfkpss = dfkpss.append(row)
            
            dftest[col,'ADF'] = dfadf
            dftest[col,'KPSS'] = dfkpss
        return dftest
        
              
# if main ...        
        
cluster=False     
#cluster=True
        

#loader = DataLoader()
if cluster:
    sample = loader.getData().sample(frac=0.01)
    Cluster_Analysis(sample)
tsa = TS_Analysis(loader.getResampledData())
# Model

In [157]:
dftest = tsa.getTestResults()
dftest

,"(requests, ADF)","(requests, KPSS)","(Temperature, ADF)","(Temperature, KPSS)","(Precipitation, ADF)","(Precipitation, KPSS)","(WindSpeed, ADF)","(WindSpeed, KPSS)"
Test Statistic,-4.821514,3.585869,-3.409897,5.917476,-9.338873e+00,0.238714,-7.526968e+00,0.707040
p-value,0.000049,0.010000,0.010626,0.010000,8.908239e-16,0.100000,3.663009e-11,0.012905
Lags Used,28.000000,28.000000,28.000000,28.000000,2.800000e+01,28.000000,2.700000e+01,28.000000
Number of Observations Used,2876.000000,NaN,2876.000000,NaN,2.876000e+03,NaN,2.877000e+03,NaN
Critical Value (1%),-3.432626,0.739000,-3.432626,0.739000,-3.432626e+00,0.739000,-3.432625e+00,0.739000
Critical Value (5%),-2.862545,0.463000,-2.862545,0.463000,-2.862545e+00,0.463000,-2.862545e+00,0.463000
Critical Value (10%),-2.567305,0.347000,-2.567305,0.347000,-2.567305e+00,0.347000,-2.567305e+00,0.347000
H0 Rejected,1.000000,1.000000,1.000000,1.000000,1.000000e+00,0.000000,1.000000e+00,1.000000


In [140]:
dftest.loc['p-value'] <= 0.05

(requests, ADF)           True
(requests, KPSS)          True
(Temperature, ADF)        True
(Temperature, KPSS)       True
(Precipitation, ADF)      True
(Precipitation, KPSS)    False
(WindSpeed, ADF)          True
(WindSpeed, KPSS)         True
Name: p-value, dtype: bool

## Original Clean Data

In [66]:
dfl = loader.getData()
# dfl = dfl.set_index('request_date')
# dfl = dfl.asfreq(freq='h')
print(len(dfl[dfl.isnull().any(axis=1)]))
dfl.head()

0


,passenger_id,source_latitude,source_longitude,source_address,destination_latitude,destination_longitude,destination_address,request_date
0,41037,-12.088156,-77.016065,"Avenida Nicolás de Arriola 314, La Victoria 13",-12.108531,-77.044891,"Calle Carlos Graña Elisande 340, San Isidro 27",2015-09-01 00:00:04
1,116591,-12.099957,-77.036497,"Av Los Conquistadores 392, San Isidro 15073",-12.119686,-76.999969,"Bruselas 228, La Calera De La Merced",2015-09-01 00:00:15
2,86426,-12.099153,-77.019425,"Av. República de Panamá 3537, San Isidro 27",-12.076505,-77.089305,"Av. La Marina cdra. 25, San Miguel 32",2015-09-01 00:00:17
3,53610,-12.110271,-77.028945,"Junín 225, Miraflores",-12.132221,-77.027021,"Calle San Fernando 380, Miraflores 18",2015-09-01 00:00:29
4,102927,-12.098430,-77.026246,"Av. República De Colombia 791, San Isidro",-12.099529,-76.990486,"Calle Mozart 201, San Borja 41",2015-09-01 00:00:31


## Resampled & Merged

In [71]:
dfl = loader.getResampledData()
# dfl = dfl.set_index('request_date')
# dfl = dfl.asfreq(freq='h')
print(len(dfl[dfl.isnull().any(axis=1)]))
#dfl.reset_index(inplace=True)
dfl.head()

0


,requests,Temperature,Precipitation,WindSpeed
request_date,,,,
2015-09-01 00:00:00,708,16.0,0.0,17.0
2015-09-01 01:00:00,479,16.0,0.0,17.0
2015-09-01 02:00:00,492,15.0,0.0,18.0
2015-09-01 03:00:00,563,15.0,0.0,18.0
2015-09-01 04:00:00,355,16.0,0.0,15.0


In [69]:
def kpss_test(timeseries):
    print ('Results of KPSS Test:')
    kpsstest = kpss(timeseries, regression='c')
    kpss_output = pd.Series(kpsstest[0:3], index=['Test Statistic','p-value','Lags Used'])
    for key,value in kpsstest[3].items():
        kpss_output['Critical Value (%s)'%key] = value
        #print (kpss_output)
    return kpss_output
    
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
        #print (dfoutput) 
    return dfoutput

In [77]:
dfkpss = kpss_test(dfl['requests'])
print(type(dfkpss))
dfkpss2 = kpss_test(dfl['requests'])
print(dfkpss)

Results of KPSS Test:
<class 'pandas.core.series.Series'>
Results of KPSS Test:
Test Statistic            3.585869
p-value                   0.010000
Lags Used                28.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


In [81]:
dfnew = pd.DataFrame()
dfnew['eleos','eleos'] = dfkpss
dfnew['eleos','eleos2'] = dfkpss2
#dfeleos = pd.merge(dfkpss, dfkpss2)
dfnew

,"(eleos, eleos)","(eleos, eleos2)"
Test Statistic,3.585869,3.585869
p-value,0.010000,0.010000
Lags Used,28.000000,28.000000
Critical Value (10%),0.347000,0.347000
Critical Value (5%),0.463000,0.463000
Critical Value (2.5%),0.574000,0.574000
Critical Value (1%),0.739000,0.739000
